# 链栈_stack

## 使用数组实现顺序栈（array_stack.py）

In [1]:
class Array(object):
    """创建链栈的数据结构"""
    def __init__(self, size=32):  # 关键属性：分配空间和存储单位（使用列表的单个元素作为一个存储单位）
        self._size = size
        self._items = [None] * size

    def __getitem__(self, index):  # Called to implement evaluation of self[index]实现下标访问.
        return self._items[index]

    def __setitem__(self, index, value):  # Called to implement assignment to self[index]. 实现下标设置值
        self._items[index] = value

    def __len__(self):
        return self._size

    def clear(self, value=None):
        """清空数据"""
        for i in range(len(self._items)):
            self._items[i] = value

    def __iter__(self):
        '''迭代器'''
        for item in self._items:
            yield item


class FullError(Exception):
    pass


class EmptyError(Exception):
    pass


class Stack:
    def __init__(self, maxsize):
        """初始化栈
        """
        self.maxsize = maxsize
        self.array = Array(maxsize)
        self.head = 0
        self.tail = 0

    def __len__(self):
        """获取栈长度"""
        return self.head - self.tail

    def push(self, value):
        """压栈：先判断是否栈满
        """
        if len(self) >= self.maxsize:
            raise FullError('stack full')
        self.array[self.head % self.maxsize] = value
        self.head += 1

    def pop(self):
        """出栈：先判断是否栈空
        """
        if len(self) <= 0:
            raise EmptyError('stack empty')
        self.head -= 1
        value = self.array[self.head % self.maxsize]
        return value

    def is_empty(self):
        """判断栈是否为空
        """
        return len(self) == 0


def test_stack():
    # 实例化栈并初始化栈元素
    s = Stack(5)
    for i in range(5):
        s.push(i)

    assert len(s) == 5
    import pytest
    with pytest.raises(FullError) as excinfo:
        s.push(5)
    assert 'full' in str(excinfo.value)
    assert s.pop() == 4
    assert s.pop() == 3
    assert s.pop() == 2
    assert s.pop() == 1
    assert s.pop() == 0

    assert s.is_empty()

    with pytest.raises(Exception) as excinfo:
        s.pop()
    assert 'empty' in str(excinfo.value)

## 栈_循环双端队列ADT（deque_stack.py）

In [2]:
class Node(object):
    """节点结构：值，前驱、后继"""
    __slots__ = ('value', 'prev', 'next')

    def __init__(self, value=None, prev=None, next=None):
        self.value, self.prev, self.next = value, prev, next


class CircularDoubleLinkedList(object):
    """循环双端链表 ADT
    循环就是把root的prev指向tail节点，串起来
    """

    def __init__(self, maxsize=None):
        """初始化循环双端队列
        最大size，前驱和后继，头节点
        """
        self.maxsize = maxsize
        node = Node()
        node.next, node.prev = node, node
        self.root = node
        self.length = 0

    def __len__(self):
        """链表的长度"""
        return self.length

    def headnode(self):
        """头节点"""
        return self.root.next

    def tailnode(self):
        """尾结点"""
        return self.root.prev

    def append(self, value):
        """链表插入：先判断是否为满"""
        if self.maxsize is not None and len(self) >= self.maxsize:  # 先看看插入的链表是否已满
            raise Exception('LinkedList is full.')
        node = Node(value=value)
        tailnode = self.tailnode()

        tailnode.next = node
        node.prev = tailnode
        node.next = self.root
        self.root.prev = node
        self.length += 1

    def appendleft(self, value):
        """左边插入"""
        if self.maxsize is not None and len(self) >= self.maxsize:
            raise Exception('LinkedList is full.')
        node = Node(value=value)

        headnode = self.headnode()
        self.root.next = node
        node.prev = self.root
        node.next = headnode
        headnode.prev = node
        self.length += 1

    def remove(self, node):
        """remove
        :param node: 传入node 而不是 value 我们就能实现 O(1) 删除
        :return:
        """
        if node is self.root:
            return
        else:
            node.prev.next = node.next
            node.next.prev = node.prev
        self.length -= 1
        return node

    def iter_node(self):
        """迭代节点：返回当前的节点"""
        if self.root.next is self.root:
            return
        curnode = self.root.next
        while curnode.next is not self.root:
            yield curnode
            curnode = curnode.next
        yield curnode

    def __iter__(self):
        """迭代节点返回当前的节点的值"""
        for node in self.iter_node():
            yield node.value

    def iter_node_reverse(self):
        """相比单链表独有的反序遍历"""
        if self.root.prev is self.root:
            return
        curnode = self.root.prev
        while curnode.prev is not self.root:
            yield curnode
            curnode = curnode.prev
        yield curnode


class DoubleEndedQueue(CircularDoubleLinkedList):
    """双端队列：继承自循环双端队列"""
    def pop(self):
        出队
        if self.tailnode() is self.root:
            raise Exception('LinkedList is empty.')
        tailnode = self.tailnode()
        value = tailnode.value
        self.remove(tailnode)
        return value

    def popleft(self):
        """左边出队"""
        if self.headnode() is self.root:
            raise Exception('LinkedList is empty.')
        headnode = self.headnode()
        value = headnode.value
        self.remove(headnode)
        return value


class Stack:
    """栈定义：初始化栈就是实例化一个双端队列
    一般的方法就是：初始化，压栈，出栈，求栈长度，判断栈空
    """
    def __init__(self):
        self.deque = DoubleEndedQueue()

    def push(self, value):
        # 压栈
        return self.deque.append(value)

    def pop(self):
        return self.deque.pop()

    def __len__(self):
        return len(self.deque)

    def is_empty(self):
        return len(self.deque) == 0


def test_stack():
    s = Stack()
    for i in range(3):
        s.push(i)

    assert len(s) == 3
    assert s.pop() == 2
    assert s.pop() == 1
    assert s.pop() == 0

    assert s.is_empty()

    import pytest
    with pytest.raises(Exception) as excinfo:
        s.pop()
    assert 'empty' in str(excinfo.value)